## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-28-08-31-37 +0000,bbc,Zelensky condemns 'vile' Russian strikes lasti...,https://www.bbc.com/news/articles/c75qeqr5905o...
1,2025-09-28-08-22-10 +0000,wapo,Four dead as massive Russian attack on Kyiv hi...,https://www.washingtonpost.com/world/2025/09/2...
2,2025-09-28-08-15-00 +0000,nypost,Maxi Doge muscles into presale — here’s how to...,https://nypost.com/business/how-to-buy-maxi-do...
3,2025-09-28-08-05-20 +0000,bbc,Selena Gomez marries music producer Benny Blanco,https://www.bbc.com/news/articles/cvgq5jzxx77o...
4,2025-09-28-08-04-40 +0000,nyt,Stampede at Actor’s Political Rally Kills at L...,https://www.nytimes.com/2025/09/27/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
241,trump,37
247,up,8
110,new,7
55,portland,7
31,political,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
31,2025-09-28-02-26-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,83
77,2025-09-27-21-15-59 +0000,wapo,"Trump calls for troops in Portland, escalating...",https://www.washingtonpost.com/politics/2025/0...,68
64,2025-09-27-22-19-00 +0000,wsj,"Trump Says He Is Ordering Troops to Portland, ...",https://www.wsj.com/us-news/trump-says-he-is-o...,66
55,2025-09-27-23-18-16 +0000,nyt,"Trump Says He Has Ordered Troops to Portland, ...",https://www.nytimes.com/2025/09/27/us/politics...,65
71,2025-09-27-21-42-04 +0000,bbc,Trump authorises 'full force' as Portland beco...,https://www.bbc.com/news/articles/cddmn6ge6e2o...,64


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
31,83,2025-09-28-02-26-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
172,40,2025-09-27-09-30-00 +0000,wsj,A new barrier for foreign workers could open s...,https://www.wsj.com/economy/jobs/trump-h1b-fee...
77,31,2025-09-27-21-15-59 +0000,wapo,"Trump calls for troops in Portland, escalating...",https://www.washingtonpost.com/politics/2025/0...
75,30,2025-09-27-21-27-07 +0000,nypost,Anti-ICE protesters led by paid activists marc...,https://nypost.com/2025/09/27/us-news/brooklyn...
21,29,2025-09-28-04-26-42 +0000,nypost,"Texas special ed teacher, ex-clown drops dead...",https://nypost.com/2025/09/28/us-news/texas-sp...
95,28,2025-09-27-18-50-03 +0000,bbc,FBI fires agents pictured kneeling at George F...,https://www.bbc.com/news/articles/cr4q5xrz9dpo...
96,27,2025-09-27-18-29-53 +0000,nypost,Israel strikes 120 targets in Gaza Strip — inc...,https://nypost.com/2025/09/27/world-news/israe...
4,26,2025-09-28-08-04-40 +0000,nyt,Stampede at Actor’s Political Rally Kills at L...,https://www.nytimes.com/2025/09/27/world/asia/...
25,23,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...
27,21,2025-09-28-03-05-31 +0000,nypost,"3 dead, 8 injured after gunman on boat opens f...",https://nypost.com/2025/09/27/us-news/north-ca...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
